In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules, fpgrowth
from mlxtend.frequent_patterns import association_rules
import pickle

In [2]:
os.getcwd() 

'C:\\Users\\a.mochalov\\Desktop\\ML\\RE_APRIORI'

In [3]:
os.chdir('C:\\Users\\a.mochalov\\Desktop\\ML\\ОПЫТЫ С АПРИОРИ')

#### Использовать кодировку encoding = "ISO-8859-1"!!! , нет!!...нужно использовать encoding='cp1251'!!! 

In [4]:
orders_12 = pd.read_csv('опыт_2_Выгрузка по чекам 01.01.2019_31.12.2019_nom_group.csv', 
                        encoding='cp1251',  header=None, sep=';', low_memory=False)

In [5]:
os.chdir('C:\\Users\\a.mochalov\\Desktop\\ML\\RE_APRIORI')      

In [6]:
# frames = [orders_11, orders_12, orders_13]
frames = [orders_12]

#### 1. Подгоотовка данных

In [8]:
orders = pd.concat(frames)

In [11]:
orders

,0,1,2,3,4,5,6
0,02/01/2019 00:00,Комплектующие и инструменты для швейных машин,М00000379,163115,L-350 (196388) Ремень капроново-кордовый,Торговля по заказам,NaN
1,02/01/2019 00:00,Спицы круговые металлические,М00000512,166931,"212182 Спицы круговые алюмин., серый цв. 60 см...",Торговля по заказам,NaN
2,02/01/2019 00:00,NaN,NaN,165541,"33263 Спицы прямые пластик 6,0мм*35см PONY",Торговля по заказам,NaN
3,02/01/2019 10:07,Лента атласная,М00000389,695807,9829 Атласная лента 50мм*25м Prym,"Владимир (ул.Егорова, 8б)",NaN
4,02/01/2019 10:08,Контейнеры для бисера и мелочей,М00000679,7700462,"930500 Контейнер для бисера, 6 секций 3,9*12,2...","Балканская пл.,д.5",89213568372
...,...,...,...,...,...,...,...
4397179,31/12/2019 19:44,"Вышивка бисером ТМ ""Созвездие""",М00000630,506488,"ЗК-001 Набор для вышивания бисером Созвездие ""...",Рыбацкое,NaN
4397180,31/12/2019 19:51,NaN,NaN,30000819,Пакет,Рыбацкое,NaN
4397181,31/12/2019 19:51,NaN,NaN,903578,"TBY.DK13701 Холст на картоне, 30*40 см, хлопок...",Рыбацкое,NaN
4397182,31/12/2019 19:51,Пакеты,М00000867,559840,3821601 Пакет ламинированный 32*44*11 см,Рыбацкое,NaN


In [12]:
orders.columns = ['Дата покупки', 'Номенклатурная группа', 'Код ном. гр', 'Артикул','Наименование', 'Подразделение', 'Телефон']

In [14]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4397184 entries, 0 to 4397183
Data columns (total 7 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   Дата покупки           object
 1   Номенклатурная группа  object
 2   Код ном. гр            object
 3   Артикул                object
 4   Наименование           object
 5   Подразделение          object
 6   Телефон                object
dtypes: object(7)
memory usage: 234.8+ MB


In [15]:
orders

,Дата покупки,Номенклатурная группа,Код ном. гр,Артикул,Наименование,Подразделение,Телефон
0,02/01/2019 00:00,Комплектующие и инструменты для швейных машин,М00000379,163115,L-350 (196388) Ремень капроново-кордовый,Торговля по заказам,NaN
1,02/01/2019 00:00,Спицы круговые металлические,М00000512,166931,"212182 Спицы круговые алюмин., серый цв. 60 см...",Торговля по заказам,NaN
2,02/01/2019 00:00,NaN,NaN,165541,"33263 Спицы прямые пластик 6,0мм*35см PONY",Торговля по заказам,NaN
3,02/01/2019 10:07,Лента атласная,М00000389,695807,9829 Атласная лента 50мм*25м Prym,"Владимир (ул.Егорова, 8б)",NaN
4,02/01/2019 10:08,Контейнеры для бисера и мелочей,М00000679,7700462,"930500 Контейнер для бисера, 6 секций 3,9*12,2...","Балканская пл.,д.5",89213568372
...,...,...,...,...,...,...,...
4397179,31/12/2019 19:44,"Вышивка бисером ТМ ""Созвездие""",М00000630,506488,"ЗК-001 Набор для вышивания бисером Созвездие ""...",Рыбацкое,NaN
4397180,31/12/2019 19:51,NaN,NaN,30000819,Пакет,Рыбацкое,NaN
4397181,31/12/2019 19:51,NaN,NaN,903578,"TBY.DK13701 Холст на картоне, 30*40 см, хлопок...",Рыбацкое,NaN
4397182,31/12/2019 19:51,Пакеты,М00000867,559840,3821601 Пакет ламинированный 32*44*11 см,Рыбацкое,NaN


In [16]:
orders.dropna(subset=['Телефон', 'Номенклатурная группа'], inplace=True)

In [17]:
# orders.drop(['ID', 'Магазин', 'Количество', 'Скидка', 'Цена', 'Сумма', 'Хар-ка', 'Код хар-ки' ], axis=1, inplace=True)

In [18]:
orders.head()

,Дата покупки,Номенклатурная группа,Код ном. гр,Артикул,Наименование,Подразделение,Телефон
4,02/01/2019 10:08,Контейнеры для бисера и мелочей,М00000679,7700462,"930500 Контейнер для бисера, 6 секций 3,9*12,2...","Балканская пл.,д.5",89213568372
6,02/01/2019 10:08,Инструменты и приспособления для изготовления...,М00000614,7709816,"Волшебный карандаш для страз, длинный,","Балканская пл.,д.5",89213568372
16,02/01/2019 10:23,Пряжа акрил 100% детская,М00000460,360051,"Пряжа Пехорка ""Детская новинка"" 50гр. 200м. (1...","Владимир (ул.Егорова, 8б)",89045902211
31,02/01/2019 11:02,Стразы термоклеевые,М00000612,7706824,"Стразы термоклеевые, 2мм, ss6, 432 шт/уп, Cris...","Владимир (ул.Егорова, 8б)",89209200543
32,02/01/2019 11:05,"Нитки ТМ ""Gutermann"" штучно и в стендах",М00000333,132057,748277 Нить для всех материалов (Sew-All) 100%...,"Балканская пл.,д.5",89217555025


In [19]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1494036 entries, 4 to 4397152
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   Дата покупки           1494036 non-null  object
 1   Номенклатурная группа  1494036 non-null  object
 2   Код ном. гр            1494036 non-null  object
 3   Артикул                1494036 non-null  object
 4   Наименование           1494036 non-null  object
 5   Подразделение          1494036 non-null  object
 6   Телефон                1494036 non-null  object
dtypes: object(7)
memory usage: 91.2+ MB


In [20]:
orders.describe(include = 'object')

,Дата покупки,Номенклатурная группа,Код ном. гр,Артикул,Наименование,Подразделение,Телефон
count,1494036,1494036,1494036,1494036,1494036,1494036,1494036
unique,202362,436,534,25044,25038,37,148176
top,17/05/2019 11:18,"Нитки ТМ ""Bestex"" штучно",М00000339,135517,"Нитки ""Bestex"" 40/2, *400ярд.",Испытателей 33,89043333024
freq,176,49523,49523,49523,49523,134702,22940


In [21]:
# orders['Номер чека'] = orders.loc[:,['Номер чека']].astype(int)

In [22]:
# сбросим индекс
orders = orders.reset_index(drop=True)

orders['Дата покупки'] = pd.to_datetime(orders['Дата покупки']).dt.date
orders

,Дата покупки,Номенклатурная группа,Код ном. гр,Артикул,Наименование,Подразделение,Телефон
0,2019-02-01,Контейнеры для бисера и мелочей,М00000679,7700462,"930500 Контейнер для бисера, 6 секций 3,9*12,2...","Балканская пл.,д.5",89213568372
1,2019-02-01,Инструменты и приспособления для изготовления...,М00000614,7709816,"Волшебный карандаш для страз, длинный,","Балканская пл.,д.5",89213568372
2,2019-02-01,Пряжа акрил 100% детская,М00000460,360051,"Пряжа Пехорка ""Детская новинка"" 50гр. 200м. (1...","Владимир (ул.Егорова, 8б)",89045902211
3,2019-02-01,Стразы термоклеевые,М00000612,7706824,"Стразы термоклеевые, 2мм, ss6, 432 шт/уп, Cris...","Владимир (ул.Егорова, 8б)",89209200543
4,2019-02-01,"Нитки ТМ ""Gutermann"" штучно и в стендах",М00000333,132057,748277 Нить для всех материалов (Sew-All) 100%...,"Балканская пл.,д.5",89217555025
...,...,...,...,...,...,...,...
1494031,2019-12-31,Инструменты для пластики,М00000750,553658,1295253 Ролик для лепки 10см акриловый,Просвещения 84,89213995500
1494032,2019-12-31,Лента атласная,М00000389,695804,9827 Атласная лента 25мм*25м Prym,"Балканская пл.,д.5",89522018155
1494033,2019-12-31,Пряжа ангорской козы ( мохер) смесовая классич...,М00000444,7728466,"Пряжа Alize ""Angora Gold Ombre Batik"" 150гр. 8...",Рыбацкое,89818125639
1494034,2019-12-31,Клеевые пистолеты,М00000712,7707496,Пистолет клеевой малый GG-010N (3K-508 ) преми...,Просвещения 84,89816874998


In [23]:
orders.columns

Index(['Дата покупки', 'Номенклатурная группа', 'Код ном. гр', 'Артикул',
       'Наименование', 'Подразделение', 'Телефон'],
      dtype='object')

#### Получим чистые транзакции по номенклатурным группам ( к ним относим 1 телефон в течение 1 ого дня, чеки не рассматриваем, т.к. они могут дробиться):

In [24]:
order_grouped = orders.groupby(['Дата покупки', 'Телефон'])['Номенклатурная группа'].apply(set)


In [25]:
order_grouped

Дата покупки  Телефон    
2019-01-02    89003649252                   {Пряжа хлопок 100% классическая  }
              89005880771    {Бусины стеклянные, хрустальные, Бумага гофрир...
              89005966917    {Пряжа ангорской козы ( мохер) смесовая класси...
              89005990908    {Пряжа шерстяная (% шерсти преобладает) смесов...
              89006002360                           {Нитки ТМ "Bestex" штучно}
                                                   ...                        
2019-12-31    89992062359    {Ножницы, Нитки ТМ «ПНК им.С.М. КИРОВА»  штучн...
              89992329201    {Ленты контактные, Тесьма отделочная разная, К...
              89992369482    {Пряжа микрополиэстр 100% фантазийная, Пенопла...
              89996354218                                      {Молнии ARTA-F}
              89996919871                                  {Рамки пластиковые}
Name: Номенклатурная группа, Length: 494051, dtype: object

In [26]:
# order_grouped_1 = orders.groupby(['Дата покупки', 'Телефон'])['Номенклатурная группа'].apply(list)
# order_grouped_1

In [27]:
df = pd.DataFrame.from_records(order_grouped)

In [28]:
df

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,Пряжа хлопок 100% классическая,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Бусины стеклянные, хрустальные",Бумага гофрированная,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Пряжа ангорской козы ( мохер) смесовая классич...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Пряжа шерстяная (% шерсти преобладает) смесова...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"Нитки ТМ ""Bestex"" штучно",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494046,Ножницы,Нитки ТМ «ПНК им.С.М. КИРОВА» штучно в стендах,"Коврики для раскроя,наборы для печворка",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
494047,Ленты контактные,Тесьма отделочная разная,Клеевые пистолеты,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
494048,Пряжа микрополиэстр 100% фантазийная,Пенопластовые заготовки объемные,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
494049,Молнии ARTA-F,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
#telephon = pd.DataFrame(order_grouped).reset_index(level=[0]).reset_index()['Телефон']
#telephon
#apriori = pd.concat([df,telephon], axis=1)
#apriori

#### 2. Визуализация данных:

#### 3. Данные подготовили, теперь Pre-Processing:

In [30]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array

transaction = []
for i in range(df.shape[0]):
    transaction.append([str(df.values[i,j]) for j in range(df.shape[1])])
    
transaction = np.array(transaction)
transaction

array([['Пряжа хлопок 100% классическая  ', 'None', 'None', ..., 'None',
        'None', 'None'],
       ['Бусины стеклянные, хрустальные', 'Бумага гофрированная', 'None',
        ..., 'None', 'None', 'None'],
       ['Пряжа ангорской козы ( мохер) смесовая классическая', 'None',
        'None', ..., 'None', 'None', 'None'],
       ...,
       ['Пряжа микрополиэстр 100% фантазийная',
        'Пенопластовые заготовки объемные', 'None', ..., 'None', 'None',
        'None'],
       ['Молнии ARTA-F', 'None', 'None', ..., 'None', 'None', 'None'],
       ['Рамки пластиковые', 'None', 'None', ..., 'None', 'None', 'None']],
      dtype='<U76')

In [31]:
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset = dataset.drop(['None'], axis=1)

In [32]:
dataset

,"Мулине ТМ ""Bestex""","Акриловые, полимерные (гипсовые) фигурки и камеи","Аксессуары для вышивания ТМ ""Hobby&Pro""","Аксессуары для вышивания ТМ ""Prym""","Аксессуары для вышивания ТМ ""Астра""",Аксессуары для вышивания разные,Аксессуары разные,Алмазная мозаика на подрамнике,"Алмазная мозаика на ткани, полная выкладка",Алмазная мозаика частичная выкладка,...,Шнуры для бижутерии,Шнуры отделочные,Шнуры силиконовые,"Штильдики, теги, вырубки",Эластичная лента упаковками,"Эластичные ленты,шнуры",Элементы из бумаги,Элементы из дерева,Элементы из картона,"яльцы и станки деревянные ТМ ""BOS"""
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494046,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
494047,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
494048,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
494049,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [33]:
dataset.loc[:, :].sum()

 Мулине ТМ "Bestex"                                  9342
Акриловые, полимерные (гипсовые) фигурки и камеи     1432
Аксессуары для вышивания  ТМ "Hobby&Pro"             1265
Аксессуары для вышивания  ТМ "Prym"                   115
Аксессуары для вышивания  ТМ "Астра"                  306
                                                    ...  
Эластичные ленты,шнуры                              14730
Элементы из бумаги                                     80
Элементы из дерева                                    190
Элементы из картона                                   437
яльцы и станки деревянные ТМ "BOS"                   1460
Length: 436, dtype: int64

In [34]:
# Convert dataset into 1-0 encoding
dataset = dataset.astype(int)

In [35]:
dataset

,"Мулине ТМ ""Bestex""","Акриловые, полимерные (гипсовые) фигурки и камеи","Аксессуары для вышивания ТМ ""Hobby&Pro""","Аксессуары для вышивания ТМ ""Prym""","Аксессуары для вышивания ТМ ""Астра""",Аксессуары для вышивания разные,Аксессуары разные,Алмазная мозаика на подрамнике,"Алмазная мозаика на ткани, полная выкладка",Алмазная мозаика частичная выкладка,...,Шнуры для бижутерии,Шнуры отделочные,Шнуры силиконовые,"Штильдики, теги, вырубки",Эластичная лента упаковками,"Эластичные ленты,шнуры",Элементы из бумаги,Элементы из дерева,Элементы из картона,"яльцы и станки деревянные ТМ ""BOS"""
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 4. Algorithm Implementation

In [36]:
# apriori

frequent_itemsets = apriori(dataset, min_support=0.0001, use_colnames=True, low_memory=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.018909,"( Мулине ТМ ""Bestex"")",1
1,0.002898,"(Акриловые, полимерные (гипсовые) фигурки и ка...",1
2,0.002560,"(Аксессуары для вышивания ТМ ""Hobby&Pro"")",1
3,0.000233,"(Аксессуары для вышивания ТМ ""Prym"")",1
4,0.000619,"(Аксессуары для вышивания ТМ ""Астра"")",1
...,...,...,...
11937,0.000101,"(Нитки ТМ ""Bestex"" штучно, Пряжа ангорской коз...",5
11938,0.000111,"(Ножницы, Пряжа ангорской козы ( мохер) смесов...",5
11939,0.000107,"(Пряжа шерсть 100% классическая, Ножницы, Пря...",5
11940,0.000101,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Ножн...",6


In [60]:
pkl_filename = "frequent_itemsets.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(frequent_itemsets, file)

In [5]:
pkl_filename = "frequent_itemsets.pkl"
with open(pkl_filename, 'rb') as file:
    frequent_itemsets = pickle.load(file)

In [6]:
for k in frequent_itemsets['itemsets']:
    print(list(k)[0])

 Мулине ТМ "Bestex"
Акриловые, полимерные (гипсовые) фигурки и камеи
Аксессуары для вышивания  ТМ "Hobby&Pro"
Аксессуары для вышивания  ТМ "Prym"
Аксессуары для вышивания  ТМ "Астра"
Аксессуары для вышивания разные 
Аксессуары разные
Алмазная мозаика на подрамнике
Алмазная мозаика на ткани, полная выкладка
Алмазная мозаика частичная выкладка
Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ
Аппликации,заплатки ТМ "Prym"
Ароматизаторы,отдушки,масла
Арт Узор
Банты
Бархат,вельвет, велюр
Бахрома
Бисер ТМ " Preciosa" 20г
Бисер ТМ " Астра"
Бисер ТМ "Preciosa" 5г
Бисероплетение ТМ "Hobby&Pro pearl"
Бисероплетение ТМ "Кроше"
Бисероплетение ТМ "Риолис"
Блочки и люверсы
Брадсы
Бубенчики,  колокольчики
Булавки английские
Булавки английские в наборах
Булавки для квилтинга
Булавки портновские
Бумага гофрированная
Бумага для квиллинга
Бумага для скрапа  в наборах
Бумага для скрапа  листами
Бумага тишью
Бусины  из смолы (имитация поделочных камней)
Бусины деревянные
Бусины металличе

Тесьма отделочная разная
Лента атласная однотонная (пр-во Китай)
Лента атласная однотонная (пр-во Китай)
Фатин на шпулях
Лента атласная однотонная (пр-во Китай)
Лента атласная однотонная (пр-во Китай)
Цветы, листья (бумажные, скелетированные)
Лента атласная однотонная (пр-во Китай)
Лента атласная однотонная (пр-во Китай)
Лента атласная однотонная (пр-во Китай)
Лента атласная однотонная (пр-во Китай)
Лента атласная с принтом
Лента атласная с принтом
Нитки ТМ "Bestex" штучно
Ножницы
Пайетки ТМ "Астра"
Лента атласная с принтом
Пуговицы костюмные
Пуговицы разного назначения
Тесьма отделочная разная
Лента атласная с принтом
Лента атласная с принтом
Ленты,тесьма техническая разная
Лента,тесьма техническая разная
Нитки ТМ "Bestex" штучно
Нитки ТМ "Gutermann"  штучно и в стендах
Ленты декоративные разные (фасовка)
Ленты капроновые,органза, для бантов
Ленты декоративные
Ленты декоративные
Ленты,тесьма техническая разная
Леска
Линейки и лекала
Ленты декоративные
Ленты декоративные
Ленты декорати

Ножницы
Ножницы
Ножницы
Пайетки ТМ "Астра"
Булавки портновские
Бусины пластиковые разные
Бусины пластиковые разные
Бусины пластиковые разные
Бусины пластиковые разные
Бусины пластиковые разные
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые (имитация жемчуга) 
Пайетки ТМ "Астра"
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые разные
Бусины пластиковые разные
Бусины пластиковые разные
Пайетки ТМ "Астра"
Бусины пластиковые разные
Бусины пластиковые разные
Шнуры отделочные
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые (имитация жемчуга) 
Контейнеры для бисера и мелочей
Леска
Бусины пластиковые (имитация жемчуга) 
Нитки ТМ "Gutermann"  штучно и в стендах
Нитки бытовой намотки  штучно разные
Ножницы
Основы для бижутерии
Пайетки ТМ "Астра"
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые (имитация жемчуга) 
Бусины пластиковые (имитация жемчуга) 
Нитки ТМ "Bestex" штучно
Нитк

Пряжа шерстяная (% шерсти преобладает) смесовая классическая
Пряжа меринос смесовая детская
Пуговицы разного назначения
Спицы круговые металлические
Пряжа шерстяная (% шерсти преобладает) смесовая классическая
Пряжа хлопок 100% классическая  
Пряжа хлопок 100% классическая  
Пряжа хлопок 100% классическая  
Пряжа хлопок 100% классическая  
Пряжа хлопок 100% классическая  
Пряжа шерстяная (% шерсти преобладает) смесовая классическая
Пуговицы костюмные
Спицы круговые металлические
Пряжа шерстяная (% шерсти преобладает) смесовая классическая
Пряжа шерстяная (% шерсти преобладает) смесовая классическая
Пуговицы костюмные
Пуговицы разного назначения
Пуговицы костюмные
Специальные иглы для вязанных изделий
Спицы круговые металлические
Эластичная лента упаковками
Пряжа ангорской козы ( мохер) смесовая классическая
Пряжа хлопок 100% классическая  
Пряжа шерстяная (% шерсти преобладает) смесовая классическая
Спицы круговые металлические
Пряжа ангорской козы ( мохер) смесовая классическая
Пряжа 

In [ ]:
frequent_itemsets['itemsets']

In [38]:
frequent_itemsets[frequent_itemsets['length']==2]

,support,itemsets,length
400,0.000265,"( Мулине ТМ ""Bestex"", Аксессуары для вышивания...",2
401,0.000227,"(Алмазная мозаика на ткани, полная выкладка, ...",2
402,0.000433,"(Аппликации, заплатки, светоотражающие наклейк...",2
403,0.000395,"(Бисер ТМ "" Preciosa"" 20г, Мулине ТМ ""Bestex"")",2
404,0.000627,"( Мулине ТМ ""Bestex"", Бисер ТМ "" Астра"")",2
...,...,...,...
5795,0.000172,"(Шнуры для бижутерии, Эластичные ленты,шнуры)",2
5796,0.000472,"(Эластичная лента упаковками, Шнуры отделочные)",2
5797,0.001757,"(Эластичные ленты,шнуры, Шнуры отделочные)",2
5798,0.000113,"(Эластичные ленты,шнуры, Шнуры силиконовые)",2


In [39]:
for k in frequent_itemsets[frequent_itemsets['length']==2]['itemsets']:
    print(k)

frozenset({' Мулине ТМ "Bestex"', 'Аксессуары для вышивания  ТМ "Hobby&Pro"'})
frozenset({'Алмазная мозаика на ткани, полная выкладка', ' Мулине ТМ "Bestex"'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"'})
frozenset({'Бисер ТМ " Preciosa" 20г', ' Мулине ТМ "Bestex"'})
frozenset({' Мулине ТМ "Bestex"', 'Бисер ТМ " Астра"'})
frozenset({' Мулине ТМ "Bestex"', 'Бисер ТМ "Preciosa" 5г'})
frozenset({' Мулине ТМ "Bestex"', 'Булавки английские'})
frozenset({'Булавки английские в наборах', ' Мулине ТМ "Bestex"'})
frozenset({' Мулине ТМ "Bestex"', 'Булавки портновские'})
frozenset({' Мулине ТМ "Bestex"', 'Бусины деревянные'})
frozenset({'Бусины пластиковые (имитация жемчуга) ', ' Мулине ТМ "Bestex"'})
frozenset({' Мулине ТМ "Bestex"', 'Бусины пластиковые (имитация стекла) '})
frozenset({' Мулине ТМ "Bestex"', 'Бусины пластиковые разные'})
frozenset({'Бусины стеклянные, хрустальные', ' Мулине ТМ "Bestex"'})
frozenset({' Мулине ТМ "Bestex

frozenset({'Пряжа акриловая (% акрила преобладает) смесовая детская', 'Ленты,тесьма техническая разная'})
frozenset({'Ленты,тесьма техническая разная', 'Пряжа акриловая (% акрила преобладает) смесовая классическая'})
frozenset({'Пряжа акриловая (% акрила преобладает) смесовая фантазийная', 'Ленты,тесьма техническая разная'})
frozenset({'Пряжа ангорской козы ( мохер) смесовая классическая', 'Ленты,тесьма техническая разная'})
frozenset({'Ленты,тесьма техническая разная', 'Пряжа меринос смесовая детская'})
frozenset({'Пряжа микрополиэстр 100% фантазийная', 'Ленты,тесьма техническая разная'})
frozenset({'Пряжа хлопок 100% классическая ', 'Ленты,тесьма техническая разная'})
frozenset({'Ленты,тесьма техническая разная', 'Пряжа хлопок 100% классическая  '})
frozenset({'Ленты,тесьма техническая разная', 'Пряжа хлопоковая (% хлопка преобладает) смесовая классическая'})
frozenset({'Ленты,тесьма техническая разная', 'Пряжа шерстяная (% шерсти преобладает) смесовая классическая'})
frozenset({'Лен

In [40]:
frequent_itemsets[frequent_itemsets['length']==3]

,support,itemsets,length
5800,0.000107,"(Ножницы, Алмазная мозаика на ткани, полная вы...",3
5801,0.000123,"(Аппликации, заплатки, светоотражающие наклейк...",3
5802,0.000101,"(Аппликации, заплатки, светоотражающие наклейк...",3
5803,0.000107,"(Аппликации, заплатки, светоотражающие наклейк...",3
5804,0.000105,"(Аппликации, заплатки, светоотражающие наклейк...",3
...,...,...,...
10201,0.000154,"(Шнуры бытовые,мебельные,шторные, Тесьма отдел...",3
10202,0.000194,"(Эластичные ленты,шнуры, Тесьма отделочная раз...",3
10203,0.000105,"(Эластичная лента упаковками, Тесьма отделочна...",3
10204,0.000130,"(Шнуры бытовые,мебельные,шторные, Эластичные л...",3


In [41]:
for k in frequent_itemsets[frequent_itemsets['length']==3]['itemsets']:
    print(k)

frozenset({'Ножницы', 'Алмазная мозаика на ткани, полная выкладка', ' Мулине ТМ "Bestex"'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Иглы  ручные швейные, специальные,наборы'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Кружево машинного производства'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Молнии ARTA-F'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Молнии разные'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Мулине ТМ " DMC"'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Нитевдеватели'})
frozenset({'Аппликации, заплатки, светоотражающие наклейки и подвески разных ТМ', ' Мулине ТМ "Bestex"', 'Нитки ТМ "Bestex"

frozenset({'Ножницы', 'Пряжа ангорской козы ( мохер) смесовая классическая', 'Нитевдеватели'})
frozenset({'Ножницы', 'Нитевдеватели', 'Пряжа меринос смесовая детская'})
frozenset({'Ножницы', 'Нитевдеватели', 'Пряжа хлопок 100% классическая  '})
frozenset({'Ножницы', 'Нитевдеватели', 'Пряжа шерстяная (% шерсти преобладает) смесовая классическая'})
frozenset({'Ножницы', 'Пуговицы блузочные', 'Нитевдеватели'})
frozenset({'Ножницы', 'Пуговицы костюмные', 'Нитевдеватели'})
frozenset({'Ножницы', 'Пуговицы пальтовые,шубные', 'Нитевдеватели'})
frozenset({'Ножницы', 'Нитевдеватели', 'Пяльцы пластиковые разных ТМ'})
frozenset({'Ножницы', 'Сантиметры,рулетки портновские', 'Нитевдеватели'})
frozenset({'Ножницы', 'Спицы двухконечные прямые (носочные) металлические', 'Нитевдеватели'})
frozenset({'Ножницы', 'Нитевдеватели', 'Спицы круговые металлические'})
frozenset({'Ножницы', 'Тесьма отделочная разная', 'Нитевдеватели'})
frozenset({'Ножницы', 'Фурнитура для белья', 'Нитевдеватели'})
frozenset({'Нож

In [42]:
frequent_itemsets[(frequent_itemsets['length'] == 4) &
                   (frequent_itemsets['support'] >= 0.000080)]

,support,itemsets,length
10206,0.000121,"( Мулине ТМ ""Bestex"", Иглы ручные швейные, сп...",4
10207,0.000107,"( Мулине ТМ ""Bestex"", Нитки ТМ ""Bestex"" штучно...",4
10208,0.000109,"(Ножницы, Мулине ТМ ""Bestex"", Иглы ручные шв...",4
10209,0.000111,"( Мулине ТМ ""Bestex"", Нитки ТМ ""Bestex"" штучно...",4
10210,0.000109,"(Ножницы, Мулине ТМ ""Bestex"", Молнии ARTA-F, ...",4
...,...,...,...
11700,0.000111,"(Эластичная лента упаковками, Пряжа ангорской ...",4
11701,0.000111,(Пряжа ангорской козы ( мохер) смесовая класси...,4
11702,0.000105,(Пряжа ангорской козы ( мохер) смесовая класси...,4
11703,0.000111,(Пряжа ангорской козы ( мохер) смесовая класси...,4


In [43]:
kk = frequent_itemsets[(frequent_itemsets['length'] == 4) &
                   (frequent_itemsets['support'] >= 0.000080)]['itemsets']

In [44]:
for k in kk:
    print(k)

frozenset({' Мулине ТМ "Bestex"', 'Иглы  ручные швейные, специальные,наборы', 'Канва фасованная белорусского производства', 'Пяльцы пластиковые разных ТМ'})
frozenset({' Мулине ТМ "Bestex"', 'Нитки ТМ "Bestex" штучно', 'Иглы  ручные швейные, специальные,наборы', 'Кружево машинного производства'})
frozenset({'Ножницы', ' Мулине ТМ "Bestex"', 'Иглы  ручные швейные, специальные,наборы', 'Кружево машинного производства'})
frozenset({' Мулине ТМ "Bestex"', 'Нитки ТМ "Bestex" штучно', 'Молнии ARTA-F', 'Иглы  ручные швейные, специальные,наборы'})
frozenset({'Ножницы', ' Мулине ТМ "Bestex"', 'Молнии ARTA-F', 'Иглы  ручные швейные, специальные,наборы'})
frozenset({' Мулине ТМ "Bestex"', 'Нитки ТМ "Bestex" штучно', 'Молнии разные', 'Иглы  ручные швейные, специальные,наборы'})
frozenset({'Ножницы', ' Мулине ТМ "Bestex"', 'Молнии разные', 'Иглы  ручные швейные, специальные,наборы'})
frozenset({'Ножницы', ' Мулине ТМ "Bestex"', 'Мулине ТМ " DMC"', 'Иглы  ручные швейные, специальные,наборы'})
frozen

In [45]:
frequent_itemsets[frequent_itemsets['length']==5]

,support,itemsets,length
11705,0.000101,"(Нитки ТМ ""Bestex"" штучно, Ножницы, Мулине ТМ...",5
11706,0.000109,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Ножн...",5
11707,0.000101,"(Молнии разные, Ножницы, Иглы ручные швейные,...",5
11708,0.000103,"(Нитки ТМ ""Bestex"" штучно, Ножницы, Иглы ручн...",5
11709,0.000101,"(Нитки ТМ ""Bestex"" штучно, Ножницы, Пряжа анго...",5
...,...,...,...
11935,0.000107,"(Нитки ТМ ""Bestex"" штучно, Ножницы, Эластичные...",5
11936,0.000103,"(Нитки ТМ ""Bestex"" штучно, Пряжа шерсть 100% ...",5
11937,0.000101,"(Нитки ТМ ""Bestex"" штучно, Пряжа ангорской коз...",5
11938,0.000111,"(Ножницы, Пряжа ангорской козы ( мохер) смесов...",5


In [46]:
frequent_itemsets[frequent_itemsets['length']==6]

,support,itemsets,length
11940,0.000101,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Ножн...",6
11941,0.000103,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Ножн...",6


In [47]:
frequent_itemsets[(frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.05)]

,support,itemsets,length


In [48]:
# fpgrowth
frequent_itemsets_fpg = fpgrowth(dataset, min_support=0.0001, use_colnames=True)
frequent_itemsets_fpg['length'] = frequent_itemsets_fpg['itemsets'].apply(lambda x: len(x))
frequent_itemsets_fpg

,support,itemsets,length
0,0.028552,(Пряжа хлопок 100% классическая ),1
1,0.014195,"(Бусины стеклянные, хрустальные)",1
2,0.003336,(Бумага гофрированная),1
3,0.053258,(Пряжа ангорской козы ( мохер) смесовая класси...,1
4,0.033875,(Пряжа шерстяная (% шерсти преобладает) смесов...,1
...,...,...,...
11937,0.000113,"(Наконечники для спиц, Спицы круговые металлич...",2
11938,0.000123,"(Грунты, Клея и лаки для декупажа)",2
11939,0.000111,"(Грунты, Краски акриловые)",2
11940,0.000152,"(Раскройные ножи и комплектующие к ним, Коврик...",2


In [49]:
frequent_itemsets_fpg[frequent_itemsets_fpg['length']==2]

,support,itemsets,length
400,0.001965,"(Пряжа хлопок 100% классическая , Пряжа акрил...",2
401,0.001269,"(Ножницы, Пряжа хлопок 100% классическая )",2
402,0.001069,"(Нитки ТМ ""Bestex"" штучно, Пряжа хлопок 100% к...",2
403,0.000974,"(Пряжа хлопок 100% классическая , Пряжа шерст...",2
404,0.001534,(Пряжа ангорской козы ( мохер) смесовая класси...,2
...,...,...,...
11937,0.000113,"(Наконечники для спиц, Спицы круговые металлич...",2
11938,0.000123,"(Грунты, Клея и лаки для декупажа)",2
11939,0.000111,"(Грунты, Краски акриловые)",2
11940,0.000152,"(Раскройные ножи и комплектующие к ним, Коврик...",2


In [50]:
frequent_itemsets_fpg[frequent_itemsets_fpg['length']==3]

,support,itemsets,length
410,0.000241,"(Ножницы, Пряжа хлопок 100% классическая , Пр...",3
411,0.000235,"(Ножницы, Пряжа ангорской козы ( мохер) смесов...",3
413,0.000213,(Пряжа ангорской козы ( мохер) смесовая класси...,3
414,0.000166,"(Нитки ТМ ""Bestex"" штучно, Пряжа хлопок 100% к...",3
415,0.000219,"(Нитки ТМ ""Bestex"" штучно, Пряжа хлопок 100% к...",3
...,...,...,...
11707,0.000125,"(Маркеры для ткани, Ножницы, Булавки портновские)",3
11828,0.000103,"(Бусины стеклянные, хрустальные, Бусины пласти...",3
11880,0.000168,"(Красители, пигменты,блестки, Формы,штампы,вк...",3
11934,0.000107,"(Красители, пигменты,блестки, Формы,штампы,вк...",3


In [51]:
frequent_itemsets_fpg[(frequent_itemsets_fpg['length'] == 4) &
                   (frequent_itemsets_fpg['support'] >= 0.000080)]

,support,itemsets,length
412,0.000128,"(Ножницы, Пряжа ангорской козы ( мохер) смесов...",4
417,0.000140,"(Ножницы, Пряжа ангорской козы ( мохер) смесов...",4
418,0.000121,(Пряжа ангорской козы ( мохер) смесовая класси...,4
419,0.000115,"(Ножницы, Нитки ТМ ""Bestex"" штучно, Пряжа хлоп...",4
420,0.000136,"(Ножницы, Нитки ТМ ""Bestex"" штучно, Пряжа хлоп...",4
...,...,...,...
10077,0.000103,"(Ножницы, Ленты,тесьма техническая разная, Нит...",4
10389,0.000103,"(Канитель, трунцал, синель, Бисер ТМ "" Астра"",...",4
10390,0.000119,"(Канитель, трунцал, синель, Бисер ТМ "" Астра"",...",4
10956,0.000101,"(Ножницы, Грейферы и наперстки, Нитевдеватели,...",4


#### Working with Sparse Representations

In [52]:
te = TransactionEncoder()
oht_ary = te.fit(transaction).transform(transaction, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
sparse_df = sparse_df.drop(['None'], axis=1)

In [53]:
frequent_sparse_df = apriori(sparse_df, min_support=0.0001, use_colnames=True, low_memory=True)
frequent_sparse_df['length'] = frequent_sparse_df['itemsets'].apply(lambda x: len(x))
frequent_sparse_df

,support,itemsets,length
0,0.018909,"( Мулине ТМ ""Bestex"")",1
1,0.002898,"(Акриловые, полимерные (гипсовые) фигурки и ка...",1
2,0.002560,"(Аксессуары для вышивания ТМ ""Hobby&Pro"")",1
3,0.000233,"(Аксессуары для вышивания ТМ ""Prym"")",1
4,0.000619,"(Аксессуары для вышивания ТМ ""Астра"")",1
...,...,...,...
11937,0.000101,"(Нитки ТМ ""Bestex"" штучно, Пряжа ангорской коз...",5
11938,0.000111,"(Ножницы, Пряжа ангорской козы ( мохер) смесов...",5
11939,0.000107,"(Пряжа шерсть 100% классическая, Ножницы, Пря...",5
11940,0.000101,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Ножн...",6


In [54]:
association_rules(frequent_sparse_df, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Ароматизаторы,отдушки,масла, Мыльные основы)","(Красители, пигменты,блестки)",0.000130,0.001069,0.000105,0.812500,760.258404,0.000105,5.327634
1,"(Формы,штампы,вкладыши, Ароматизаторы,отдушки,...","(Красители, пигменты,блестки)",0.000134,0.001069,0.000107,0.803030,751.397584,0.000107,5.071497
2,"(Тесьма шторная, Мулине ТМ "" DMC"")","(Нитки ТМ ""Bestex"" штучно)",0.000136,0.053347,0.000103,0.761194,14.268807,0.000096,3.964111
3,"( Мулине ТМ ""Bestex"", Иглы ручные швейные, сп...","(Нитки ТМ ""Bestex"" штучно)",0.000150,0.053347,0.000107,0.716216,13.425684,0.000099,3.335826
4,"(Ножницы, Мулине ТМ ""Bestex"", Кружево машинно...","(Иглы ручные швейные, специальные,наборы)",0.000156,0.023087,0.000109,0.701299,30.376760,0.000106,3.270536
...,...,...,...,...,...,...,...,...,...
1576,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Ножн...",(Пряжа ангорской козы ( мохер) смесовая класси...,0.000146,0.053258,0.000103,0.708333,13.300121,0.000095,3.245974
1577,"(Нитки ТМ ""Bestex"" штучно, Молнии разные, Пряж...",(Ножницы),0.000121,0.039686,0.000103,0.850000,21.418032,0.000098,6.402092
1578,"(Нитки ТМ ""Bestex"" штучно, Ножницы, Пряжа анго...",(Молнии разные),0.000125,0.039259,0.000103,0.822581,20.952608,0.000098,5.415085
1579,"(Молнии разные, Ножницы, Пряжа ангорской козы ...","(Нитки ТМ ""Bestex"" штучно)",0.000123,0.053347,0.000103,0.836066,15.672296,0.000097,5.774585
